In [3]:
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt
import seaborn as sns
from ultralytics import YOLO

In [12]:
Directory = r"F:\Work\SOCAAutomation\Dataset\Images"#\09072024_1_1"
FileName = "1.jpg"
img = cv.imread(Directory + "/" + FileName)

In [5]:
model = YOLO('best.pt')

In [16]:
results = model(img, conf=0.3, save=True, hide_labels=True)

WARNING  'hide_labels' is deprecated and will be removed in 'ultralytics 8.2' in the future. Please use 'show_labels' instead.

0: 544x640 2 Fire-Smoke-Sodium-Trays, 4306.9ms
Speed: 38.0ms preprocess, 4306.9ms inference, 130.1ms postprocess per image at shape (1, 3, 544, 640)
Results saved to runs\segment\predict2


In [14]:
results[0].masks

ultralytics.engine.results.Masks object with attributes:

data: tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]])
orig_shape: (576, 704)
shape: torch.Size([2, 544, 640])
xy: [array([[      304.7,         376],
       [      303.6,       377.1],
       [      303.6,       378.2],
       [      301.4,       380.4],
       [      300.3,       380.4],
       [      299.2,       381.5],
       [      299.2,       382.6],
       [        297,       384.8],
       [      295.9,       384.8],
       [      291.5,       389.2],
 

In [15]:
results

[ultralytics.engine.results.Results object with attributes:
 
 boxes: ultralytics.engine.results.Boxes object
 keypoints: None
 masks: ultralytics.engine.results.Masks object
 names: {0: 'Fire', 1: 'Fire-Smoke-Sodium-Tray'}
 orig_img: array([[[  3,   1,   1],
         [  3,   1,   1],
         [  0,   0,   1],
         ...,
         [ 98,  89,  76],
         [ 26,  16,   6],
         [ 11,   2,   0]],
 
        [[  2,   0,   0],
         [  2,   0,   0],
         [  0,   0,   1],
         ...,
         [ 97,  87,  77],
         [ 25,  16,   6],
         [  9,   3,   0]],
 
        [[  0,   0,   0],
         [  0,   0,   0],
         [  1,   1,   1],
         ...,
         [ 96,  86,  76],
         [ 23,  16,   7],
         [  7,   2,   0]],
 
        ...,
 
        [[  0,   0,   0],
         [  0,   0,   0],
         [  0,   0,   0],
         ...,
         [169, 173, 197],
         [ 30,  31,  51],
         [  0,   0,  13]],
 
        [[  0,   0,   0],
         [  0,   0,   0],
       